In [3]:
import cv2
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import json

In [4]:
def make_rectangle(img: np.array, js: dict) -> np.array:
    # 사각형 좌표 구하기
    top,bot = (js['top_x'],js['top_y']),(js['bot_x'],js['bot_y'])

    # 사각형 그리기
    cv2.rectangle(img,top,bot,(255,255,255)) # Grayscale 이라서 흰색으로 그려야됨
    
    return img

def otsu_algorithm(img: np.array) -> np.array:
    ret, otsu = cv2.threshold(img,-1,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return otsu

In [5]:
## TrainData 
train_json_path = './dataset/train/train_defected_json'
train_img_path = './dataset/train/train_defected_dataset'

# make dir
if not os.path.isdir('./dataset/Bboximages/train'):
    os.makedirs('./dataset/Bboximages/train/st')
    os.makedirs('./dataset/Bboximages/train/aq')
    os.makedirs('./dataset/Bboximages/train/fl')

for idx in os.listdir(train_json_path):
    # json files load
    with open(f'{train_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image
    img = cv2.imread(f'{train_img_path}/{json_dict["image_name"]}', cv2.IMREAD_GRAYSCALE)

    # apply otsu algorithm
    img = otsu_algorithm(img)

    # draw rectangle
    make_rectangle(img, json_dict)

    # save image
    cv2.imwrite(f'./dataset/Bboximages/train/{json_dict["defect_class"]}/{json_dict["image_name"]}',img)

In [12]:
## Validation data 
valid_json_path = './dataset/validation/validation_defected_json'
valid_img_path = './dataset/validation/validation_defected_dataset'

# make dir
if not os.path.isdir('./dataset/Bboximages/validation'):
    os.makedirs('./dataset/Bboximages/validation/st')
    os.makedirs('./dataset/Bboximages/validation/aq')
    os.makedirs('./dataset/Bboximages/validation/fl')

for idx in os.listdir(valid_json_path):
    # json files load
    with open(f'{valid_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image
    img = cv2.imread(f'{valid_img_path}/{json_dict["image_name"]}', cv2.IMREAD_GRAYSCALE)

    # apply otsu algorithm
    img = otsu_algorithm(img)

    # draw rectangle
    make_rectangle(img, json_dict)

    # save image
    cv2.imwrite(f'./dataset/Bboximages/validation/{json_dict["defect_class"]}/{json_dict["image_name"]}',img)

In [41]:
# Bbox의 모든 픽셀 값을 더해서 검출 되는지 안되는지 확인
# 이진 알고리즘 돌렸을때 검출이 된다면 Bbox 내에 반드시 흰색이 있을것이다 라는 가정
def ROI_sum(img: np.array,js: dict) -> int:
    img = img[js['top_y']:js['bot_y'], js['top_x']:js['bot_x']]
    return sum(sum(img))

vaild_defected_fail = []
train_defected_fail = []

# train 데이터에서 Bbox 체크
for idx in os.listdir(train_json_path):
    # json files load
    with open(f'{train_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image
    img = cv2.imread(f'{train_img_path}/{json_dict["image_name"]}', cv2.IMREAD_GRAYSCALE)

    # apply otsu algorithm
    img = otsu_algorithm(img)

    # sum of all pixels
    pixel = ROI_sum(img, json_dict)

    # check defect
    if not pixel:
        train_defected_fail.append(json_dict['image_name'])

# validation 데이터에서 Bbox 체크 
for idx in os.listdir(valid_json_path):
    # json files load
    with open(f'{valid_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image
    img = cv2.imread(f'{valid_img_path}/{json_dict["image_name"]}', cv2.IMREAD_GRAYSCALE)

    # apply otsu algorithm
    img = otsu_algorithm(img)

    # sum of all pixels
    pixel = ROI_sum(img, json_dict)

    # check defect
    if not pixel:
        vaild_defected_fail.append(json_dict['image_name'])

In [44]:
len(vaild_defected_fail),len(train_defected_fail) # 124 515
(124+515)/3750

0.1704